In [114]:
import pandas as pd
import numpy as np

In [115]:
data = pd.read_csv("Harvey_stressor_with_topics.csv")

topics = [19,25,29,48,52]
extracted_data = data.loc[data["Topic_number"].isin(topics)]
extracted_data = extracted_data.iloc[:,3:]
extracted_data["Time"] = extracted_data["Time"].astype('datetime64[ns]')
extracted_data["Date"] = [d.date() for d in extracted_data["Time"]]
extracted_data["Timestamp"] = [d.time() for d in extracted_data["Time"]]

# define the bins
bins = [0, 6, 12, 18, 24]

# add custom labels if desired
labels = ['00:00:00', '06:00:00', '12:00:00', '18:00:00']

# Binning the time
extracted_data['Time Bin'] = pd.cut(extracted_data["Time"].dt.hour, bins, labels=labels, right=False)

extracted_data

,ID,Likes,Replies,Retweets,Time,Tweet,Emotion,Topic_number,Date,Timestamp,Time Bin
1,9.010000e+17,0.0,0.0,0.0,2017-08-25 14:44:00,If youre able give blood at RedCrossDonate blo...,disapproval,48,2017-08-25,14:44:00,12:00:00
5,9.010000e+17,0.0,0.0,0.0,2017-08-25 14:44:00,Hi Maria Will Bush AirportHobby Expect SuperSt...,curiosity,19,2017-08-25,14:44:00,12:00:00
8,9.010000e+17,1.0,0.0,0.0,2017-08-25 14:44:00,To everyone in TexasPlease get to safety and i...,caring,19,2017-08-25,14:44:00,12:00:00
9,9.010000e+17,37.0,2.0,21.0,2017-08-25 14:44:00,Texans Told to Evacuate Ahead of Hurricane Har...,caring,48,2017-08-25,14:44:00,12:00:00
35,9.010000e+17,4.0,0.0,4.0,2017-08-25 14:45:00,Are you tech ready for hurricaneHurricanePrep ...,curiosity,48,2017-08-25,14:45:00,12:00:00
...,...,...,...,...,...,...,...,...,...,...,...
98156,9.030000e+17,0.0,0.0,0.0,2017-08-29 13:52:00,Why we wont be ready for the next Hurricane Ha...,confusion,48,2017-08-29,13:52:00,12:00:00
98204,9.030000e+17,0.0,0.0,0.0,2017-08-29 13:55:00,Houstons hotel market already bad is poised to...,disappointment,19,2017-08-29,13:55:00,12:00:00
98205,9.030000e+17,0.0,0.0,0.0,2017-08-29 13:55:00,Give wisely in the wake of Hurricane Harveys d...,caring,48,2017-08-29,13:55:00,12:00:00
98208,9.030000e+17,0.0,0.0,0.0,2017-08-29 13:55:00,Watching footage and looking at photo of all o...,sadness,19,2017-08-29,13:55:00,12:00:00


In [116]:
topic_dist = extracted_data["Topic_number"].value_counts()
topic_dist

Topic_number
29    1851
52    1630
19    1616
25    1548
48    1227
Name: count, dtype: int64

In [117]:
dash_data = extracted_data.groupby(['Date','Time Bin','Topic_number'], as_index=False).size()
dash_data["Time"] = dash_data['Date'].astype(str) + ' ' + dash_data['Time Bin'].astype(str)
dash_data["Time"] = dash_data["Time"].astype('datetime64[ns]')
dash_data["Topic_number"] = dash_data["Topic_number"].astype(str)

dash_data.columns.values[3] = "count"
dash_data

,Date,Time Bin,Topic_number,count,Time
0,2017-08-22,00:00:00,19,0,2017-08-22 00:00:00
1,2017-08-22,00:00:00,25,0,2017-08-22 00:00:00
2,2017-08-22,00:00:00,29,0,2017-08-22 00:00:00
3,2017-08-22,00:00:00,48,0,2017-08-22 00:00:00
4,2017-08-22,00:00:00,52,0,2017-08-22 00:00:00
...,...,...,...,...,...
155,2017-08-29,18:00:00,19,0,2017-08-29 18:00:00
156,2017-08-29,18:00:00,25,0,2017-08-29 18:00:00
157,2017-08-29,18:00:00,29,0,2017-08-29 18:00:00
158,2017-08-29,18:00:00,48,0,2017-08-29 18:00:00


In [118]:
dash_data = dash_data.iloc[:,2:]
data = dash_data.pivot_table(index=['Time'], columns='Topic_number',values="count").reset_index()
data["Time"] = data["Time"].astype('datetime64[ns]')
data.to_csv("dashboard_data.csv",index=False)

In [124]:
import plotly.graph_objects as go
import pandas as pd


data = pd.read_csv("dashboard_data.csv")
layout = go.Layout(
    autosize=False,
    width=1600,
    height=640,
    xaxis=go.layout.XAxis(linecolor="black", linewidth=1, mirror=True),
    yaxis=go.layout.YAxis(linecolor="black", linewidth=1, mirror=True),
    margin=go.layout.Margin(l=50, r=50, b=100, t=100, pad=4),
)


fig = go.Figure(layout= layout)

fig.add_trace(go.Scatter(x=data["Time"], y=data["19"], fill='tozeroy',name="Topic 19",
                        hovertemplate ='tweet count: %{y}\n'+'Time: %{x}'))
fig.add_trace(go.Scatter(x=data["Time"], y=data["25"], fill='tozeroy',name="Topic 25",
                        hovertemplate ='tweet count: %{y}\n'+'Time: %{x}'))
fig.add_trace(go.Scatter(x=data["Time"], y=data["29"], fill='tozeroy',name="Topic 29",
                        hovertemplate ='tweet count: %{y}\n'+'Time: %{x}'))
fig.add_trace(go.Scatter(x=data["Time"], y=data["48"], fill='tozeroy',name="Topic 48",
                        hovertemplate ='tweet count: %{y}\n'+'Time: %{x}')) 
fig.add_trace(go.Scatter(x=data["Time"], y=data["52"], fill='tozeroy',name="Topic 52",
                        hovertemplate ='tweet count: %{y}\n'+'Time: %{x}'))

fig.update_xaxes(
    dtick=21600000,
    tickformat="%H:%M \n %b %d",
    ticklabelmode="period",
    ticks="inside",

)


fig.show()
fig.write_html("file.html")

In [120]:
# from dash import Dash, dcc, html, Input, Output
# import plotly.express as px

# app = Dash(__name__)


# app.layout = html.Div([
#     html.H4('Test analysis'),
#     dcc.Graph(id="time-series-chart"),
#     html.P("Select Topic:"),
#     dcc.Dropdown(
#         id="ticker",
#         options=[19,25,29,48,52],
#         value=19,
#         clearable=False,
#     ),
# ])


# @app.callback(
#     Output("time-series-chart", "figure"), 
#     Input("ticker", "value"))
# def display_time_series(ticker):
#     df = data # replace with your own data source
#     fig = px.area(df, x='Time', y=ticker)
#     return fig


# app.run_server(debug=True)